<a href="https://colab.research.google.com/github/CPukszta/gitbrachmerge/blob/main/homework/hw2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up Google Colab data path

In [3]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade iqplot bebi103 watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"
# ------------------------------

Import statements

In [49]:
import numpy as np
import pandas as pd

import bokeh.io
bokeh.io.output_notebook()

Load the data set. Note that it is already tidy

In [5]:
df = pd.read_csv(os.path.join(data_path, "ant_joint_locations.zip"), comment='#')

df.head()

,frame,beetle_treatment,ID,bodypart,x_coord,y_coord,likelihood
0,0,dalotia,0,head,73.086,193.835,1.0
1,1,dalotia,0,head,73.730,194.385,1.0
2,2,dalotia,0,head,75.673,195.182,1.0
3,3,dalotia,0,head,77.319,196.582,1.0
4,4,dalotia,0,head,78.128,197.891,1.0


**Part A)** Create the columns 'x (mm)', 'y (mm)', and 'time (sec)' since recording started using the information that interpixel distance is 0.8mm and there are 28 frames/second.

In [6]:
df['x (mm)'] = df['x_coord'] * 0.8
df['y (mm)'] = df['y_coord'] * 0.8
df['time (s)'] = df['frame'] / 28.0
df.head()

,frame,beetle_treatment,ID,bodypart,x_coord,y_coord,likelihood,x (mm),y (mm),time (s)
0,0,dalotia,0,head,73.086,193.835,1.0,58.4688,155.0680,0.000000
1,1,dalotia,0,head,73.730,194.385,1.0,58.9840,155.5080,0.035714
2,2,dalotia,0,head,75.673,195.182,1.0,60.5384,156.1456,0.071429
3,3,dalotia,0,head,77.319,196.582,1.0,61.8552,157.2656,0.107143
4,4,dalotia,0,head,78.128,197.891,1.0,62.5024,158.3128,0.142857


Part B) to display position over time of the ants depending on beetle treatment, we will group the ants by beetle condition, then select only the thorax data.

In [21]:
df_dalotia = df.loc[df['beetle_treatment'] == "dalotia"]
df_dalotia_thorax = df_dalotia.loc[df_dalotia['bodypart'] == 'thorax']
df_dalotia_thorax.head()

df_sc = df.loc[df['beetle_treatment'] == "sceptobius"]
df_sc_thorax = df_sc.loc[df_sc['bodypart'] == 'thorax']
df_sc_thorax.head()

,frame,beetle_treatment,ID,bodypart,x_coord,y_coord,likelihood,x (mm),y (mm),time (s)
309969,0,sceptobius,6,thorax,55.068,68.461,1.0,44.0544,54.7688,0.000000
309970,1,sceptobius,6,thorax,54.850,69.484,1.0,43.8800,55.5872,0.035714
309971,2,sceptobius,6,thorax,53.124,70.773,1.0,42.4992,56.6184,0.071429
309972,3,sceptobius,6,thorax,52.320,74.373,1.0,41.8560,59.4984,0.107143
309973,4,sceptobius,6,thorax,51.685,73.973,1.0,41.3480,59.1784,0.142857


Part B) Data is already grouped by beetle treatment. Now going to add a new column that computes the Euclidean distance between each time step before summing all of the distances travelled by each ant (grouping by ID)

In [22]:
df_dalotia_thorax['distance step'] = np.zeros(len(df_dalotia_thorax['frame']))

for ind in df_dalotia_thorax.index:
  if (df_dalotia_thorax['frame'][ind] == 0):
    df_dalotia_thorax['distance step'][ind] = 0.0
  else:
    pt_prev = np.array(df_dalotia_thorax['x (mm)'][(ind-1)], df_dalotia_thorax['y (mm)'][(ind-1)])
    pt_curr = np.array(df_dalotia_thorax['x (mm)'][(ind)], df_dalotia_thorax['y (mm)'][(ind)])
    df_dalotia_thorax['distance step'][ind] = np.abs(np.linalg.norm(pt_prev - pt_curr))

df_sc_thorax['distance step'] = np.zeros(len(df_sc_thorax['frame']))

for ind in df_sc_thorax.index:
  if (df_sc_thorax['frame'][ind] == 0):
    df_sc_thorax['distance step'][ind] = 0.0
  else:
    pt_prev = np.array(df_sc_thorax['x (mm)'][(ind-1)], df_sc_thorax['y (mm)'][(ind-1)])
    pt_curr = np.array(df_sc_thorax['x (mm)'][(ind)], df_sc_thorax['y (mm)'][(ind)])
    df_sc_thorax['distance step'][ind] = np.abs(np.linalg.norm(pt_prev - pt_curr))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

Part B) going to group by ID in order to see total distance summed by each ant

In [45]:
df_sc_thorax["cumulative distance"] = np.zeros(len(df_sc_thorax["frame"]))
for unique_ID in df_sc_thorax['ID'].unique():
  inds = (df_sc_thorax['ID'] == unique_ID)
  df_sc_thorax["cumulative distance"][inds] = df_sc_thorax.loc[inds, 'distance step'].cumsum()

df_dalotia_thorax["cumulative distance"] = np.zeros(len(df_dalotia_thorax["frame"]))
for unique_ID in df_dalotia_thorax['ID'].unique():
  inds = (df_dalotia_thorax['ID'] == unique_ID)
  df_dalotia_thorax["cumulative distance"][inds] = df_dalotia_thorax.loc[inds, 'distance step'].cumsum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

Plotting our cumulative distance vs time for the ants with both species of beatles

In [70]:
colors = ["#006837","#c7e9b4","#7fcdbb","#41b6c4","#2c7fb8","#253494"]
colors_2 = ["#d4b9da","#c994c7","#df65b0","#dd1c77","#980043","#7a0177"]

p = bokeh.plotting.figure(
    height = 600,
    width = 600,
    x_axis_label = "Time (s)",
    y_axis_label = "Cumulative Distance (mm)",
    title = "Cumulative Distance Over Time"
)

for i, g in df_dalotia_thorax.groupby('ID'):
  p.circle(g["time (s)"],g["cumulative distance"], color=colors[i],legend_label="Dalotia")

for i, g in df_sc_thorax.groupby('ID'):
  p.circle(g["time (s)"],g["cumulative distance"], color=colors_2[i % 6],legend_label="Sceptobius")


bokeh.io.show(p)

Watermark

In [10]:
%load_ext watermark
%watermark -v -p pandas,jupyterlab

Python implementation: CPython
Python version       : 3.7.14
IPython version      : 7.9.0

pandas    : 1.3.5
jupyterlab: not installed

